In [3]:
import os, random
import numpy as np
import pandas as pd
from tqdm import tqdm  
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import sklearn.metrics
from typing import Dict, List, Optional
import albumentations as A
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import rasterio
import ttach as tta

In [35]:
class Cfg:
    model_name             = "maxvit_tiny_tf_512.in1k" # tf_efficientnet_b0, convnext_atto, convnextv2_nano.fcmae_ft_in22k_in1k_384
    img_size               = 512  # 128, 512
    num_classes            = 1
    in_channels            = 1    # 1, 13
    batch_size             = 8
    drop_rate              = 0.0
    drop_path_rate         = 0.0
    iters_to_accumulate    = 1
    grad_clip              = None # None

    device                 = "cuda"
    use_amp                = False   #? bfloat?16
    compile                = False
    num_workers            = 4
    exp_name               = "exp1" 
    seed                   = 0

In [36]:
class NetDataset(Dataset):
    def __init__(self, df, cfg, transform):
        super().__init__()
        self.cfg = cfg
        self.df = df # np.array
        self.transform = transform
        
        self.val_transforms = A.Compose(
            [   
                # A.Resize(cfg.img_size, cfg.img_size),
                ToTensorV2(),
            ]
        )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        with rasterio.open("test_images/" + row.filename) as img:
            x = img.read().astype(np.float32)  #[1:4] [[1,2,3,4]]

        x = np.concatenate((x, x[1:4]))
        img_rows = []
        for min_id in range(0, 16, 4):
            img_row = np.hstack([i for i in x[min_id:min_id+4]])
            img_rows.append(img_row) #.transpose(2, 1, 0)
        x = np.vstack(img_rows)

        x = np.where(x < 0, 0, x)
        max, min = 2.8003, 0.0
        x = (x - min) / (max - min + 1e-8)

        # x = x.transpose(1,2,0)
        transformed = self.val_transforms(image=x)
        x = transformed["image"]
        # x = x.transpose(1,0) 
        # x = torch.unsqueeze(x, dim=0)

        return x

In [37]:
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(self.cfg.model_name, pretrained=True, num_classes=self.cfg.num_classes, 
                                       in_chans=self.cfg.in_channels, drop_rate=self.cfg.drop_rate, 
                                       drop_path_rate=self.cfg.drop_path_rate)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [38]:
def Datasetloader(df, cfg):
    validloader = None
        
    valid_dataset = NetDataset(df, cfg, "val")
    validloader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, 
                             num_workers=cfg.num_workers, pin_memory=True, drop_last=False)

    return validloader

In [39]:
def eval_model(model, validloader, cfg):
    preds = []

    transforms = tta.Compose(
        [
            # tta.HorizontalFlip(),
            # tta.VerticalFlip(),
            # tta.Rotate90(angles=[0, 180]),
            # tta.Scale(scales=[1, 2, 4]),
            # tta.Multiply(factors=[0.9, 1, 1.1]),        
        ]
    )
    model = tta.ClassificationTTAWrapper(model, transforms)

    model.eval()
    for (x) in tqdm(validloader): #tqdm
        x = x.to(cfg.device)
        with torch.inference_mode():  
            with torch.autocast(device_type=cfg.device, dtype=torch.float16, enabled=cfg.use_amp):
                logits = model(x)#.squeeze()

        preds.append(logits.detach().cpu())

    preds = torch.cat(preds)

    return preds

In [ ]:
cfg = Cfg()

if cfg.use_amp == False:
    torch.set_float32_matmul_precision('high')

df = pd.read_csv("sample_answer.csv")

validloader = Datasetloader(df, cfg)

model_paths = [
                # "1_convnext_atto_3e4_L1_tta_fold0_epoch20.pt", 
                #    "convnext_atto_3e4_L1_tta_fold0_epoch20.pt",
                #    "convnext_atto_40_3e4_L1_tta_fold0_epoch20.pt",
                # "convnext_atto_3e4_MSE_tta_fold0_epoch20.pt", # 0.9790
                # "convnext_atto_3e4_L1_bunchaaug_tta_fold0_epoch20.pt", # 0.9855
                # "convnext_atto_3e4_L1_tta_4x4_fold0_epoch20.pt", # 0.9828
                # "maxvit_tiny_tf_384.in1k_1e4_L1_tta_fold0_epoch20.pt", # 0.9863
                # "convnext_atto_16b_1e4_L1_tta_fold0_epoch20.pt", # 0.984
                # "maxvit_tiny_tf_512.in1k_4x4_3e4_L1_tta_bs8_fold0_epoch20.pt", # 0.9896
                # "maxvit_tiny_tf_512.in1k_4x4_3e4_L1_tta_bs8_dr0.1_fold0_epoch20.pt", # 0.9774
                # "maxvit_tiny_tf_512.in1k_4x4_1e4_L1_tta_bs8_dr01_fold0_epoch20.pt,   # 0.9884
                # "convnext_atto_4x4_3e4_L1_bs16_dr01_fold0_epoch20.pt",
                # "convnext_atto_4x4_3e4_L1_tta_bs16_dr01_fold0_epoch40.pt",
                # "maxvit_tiny_tf_384.in1k_3e4_L1_tta_bs8_384_fold0_epoch20.pt",
                 "maxvit_tiny_tf_512.in1k_3e4_L1_tta_bs8_384_fold0_epoch20.pt",
               ]

ensembled = None
for i in range(len(model_paths)):
    model = Net(cfg).to(cfg.device) 
    model.load_state_dict(torch.load("models/" + model_paths[i]))
    pred = eval_model(model, validloader, cfg)
    if ensembled is None:
        ensembled = pred
    else:
        ensembled += pred

preds = ensembled / len(model_paths)

df.target = preds
df.to_csv("submission.csv", index=False, header=False)

100%|██████████| 187/187 [00:09<00:00, 18.92it/s]


In [43]:
subs = ["submissions/maxvit_0.9896_sub.csv",
        "submissions/0.9884_maxvit.csv",
        # "submissions/sub_convnext_atto_4x4_3e4_L1_bs16_dr01_fold0_epoch20.csv",
        # "submissions/sub_convnext_atto_4x4_3e4_L1_bs16_dr01_fold0_epoch40.csv",
        # "submissions/sub_maxvit_tiny_tf_384.in1k_3e4_L1_tta_bs8_384_fold0_epoch20.csv",
        "submissions/sub_maxvit_tiny_tf_512.in1k_3e4_L1_tta_bs8_384_fold0_epoch20.csv",
        ]

ensembled = None
for i in range(len(subs)):
    if ensembled is None:
        ensembled = pd.read_csv(subs[i]).pred
    else:
        ensembled += pd.read_csv(subs[i]).pred

ensembled = ensembled / len(subs)
ensembled = pd.concat([df.filename, ensembled], axis=1)
ensembled

,filename,pred
0,test_1.tif,0.081642
1,test_2.tif,0.199914
2,test_3.tif,0.211495
3,test_4.tif,0.110665
4,test_5.tif,0.085977
...,...,...
1484,test_1485.tif,0.096455
1485,test_1486.tif,0.171889
1486,test_1487.tif,0.159528
1487,test_1488.tif,0.103038


In [44]:
ensembled.to_csv("ensemble_sub.csv", index=False, header=False)